# 기본 설정

구글 드라이브 마운트

패키지 설치

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
!pip install transformers
!pip install datasets
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 23.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-no

In [3]:
import pandas as pd
import numpy as np
from glob import glob
import os
import tqdm
from transformers import (
    AutoTokenizer,
    AutoModel,
    DataCollatorWithPadding
)

from datasets import Dataset, Value
import torch
import torch.nn as nn
from typing import List, Optional, Tuple, Union
import datetime
import re

In [4]:
#@title 마이너 패키지 로딩
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

<ipython-input-4-c31c0f4e8541>:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


In [12]:
cd /gdrive/MyDrive/Lectures/2024/RecSys/content-based

/gdrive/MyDrive/Lectures/2024/RecSys/content-based


# 데이터 로드

`../raw/ml-100k`
- u1.base: 학습 (80K)
- u1.test: 검증 (20K)

In [13]:
#@title get_user_item_map
def get_user_item_map(X):
  """Function to generate a ratings matrx and mappings for
  the user and item ids to the row and column indices

  Parameters
  ----------
  X : pandas.DataFrame, shape=(n_ratings,>=3)
      First 3 columns must be in order of user, item, rating.

  Returns
  -------
  user_map : pandas Series, shape=(n_users,)
      Mapping from the original user id to an integer in the range [0,n_users)
  item_map : pandas Series, shape=(n_items,)
      Mapping from the original item id to an integer in the range [0,n_items)
  """
  user_col, item_col, rating_col = X.columns[:3]
  rating = X[rating_col]
  user_map = pd.Series(
      index=np.unique(X[user_col]),
      data=np.arange(X[user_col].nunique()),
      name='user_map',
  )
  item_map = pd.Series(
      index=np.unique(X[item_col]),
      data=np.arange(X[item_col].nunique()),
      name='columns_map',
  )

  return user_map, item_map

In [14]:
item_plot_df = pd.read_csv('movie_plots_80_missings.csv', index_col=0)

In [15]:
def load_data(file_path):
  ratings_df = pd.read_csv(file_path, sep='\t', header=None,
                          names=['userId', 'movieId', 'rating', 'timestamp'])
  ratings_df['timestamp'] = ratings_df['timestamp'].apply(datetime.fromtimestamp)
  ratings_df = ratings_df.sort_values('timestamp')
  return ratings_df

In [16]:
train_df = load_data('../raw/ml-100k/u1.base').reset_index(drop=True)
val_df = load_data('../raw/ml-100k/u1.test').reset_index(drop=True)

In [17]:
user_map, item_map = get_user_item_map(pd.concat((train_df, val_df), axis=0))

In [18]:
train_df['plots'] = train_df.movieId.map(item_plot_df['plot'])
val_df['plots'] = val_df.movieId.map(item_plot_df['plot'])

# 모델링

컨텐츠 모델링
- 플롯(plot) 활용: Distilbert (https://huggingface.co/docs/transformers/model_doc/distilbert#distilbert)

유저 모델링
- 단순 임베딩

결합
- 연결: concatenation

예측
- Linear regression

아이템 플롯 데이터
- movie_plots_80_missings.csv

참고
- ~~https://ratsgo.github.io/nlpbook/docs/lm~~
- https://huggingface.co/docs/transformers/index


In [19]:
pretrained_model_tag = "distilbert-base-uncased"

## 데이터셋

In [20]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_tag)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [21]:
def preprocess_function(examples):
    return tokenizer(examples["plots"], truncation=True, max_length=256)

데이터셋 변환
- `Dataset.map(fn)`

In [22]:
def make_dataset(df):
    ds = Dataset.from_pandas(pd.concat([df.userId.map(user_map),
                              df.movieId.map(item_map),
                              df.rating,
                              df.plots],
                             axis=1)
                            )
    ds = ds.map(preprocess_function, batched=True)
    ds = ds.rename_column("userId", "user_id")
    ds = ds.rename_column("movieId", "movie_id")
    ds = ds.rename_column("rating", "label")
    ds = ds.cast_column('label', Value("float"))
    return ds.remove_columns(['plots', 'movie_id'])

토크나이징

In [23]:
train_ds = make_dataset(train_df)
validation_ds = make_dataset(val_df)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/20000 [00:00<?, ? examples/s]

포맷 변경
- `torch` 포맷으로...

In [24]:
train_ds = train_ds.with_format('torch').shuffle(
        seed=0
    )
validation_ds = validation_ds.with_format('torch')

## DataCollatorWithPadding

모델에 전달할 최종 입력과 출력 가공
- 만약 입력이 B개의 미니 배치로 구성될 때
- 각 토큰 시퀀스인 `input_ids`의 길이가 다르다면
- 끝을 패딩하여, 미니 배치 내의 모든 시퀀스의 길이가 동일하게 해주는 역할
- (필요시 출력도 변경...)

In [25]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### DataCollator 테스트

In [26]:
inputs = train_ds[:16]

inputs_len = [len(x) for x in inputs['input_ids']]

# 입력 시퀀스의 길이 조사
print(inputs_len)

# 문장들을 배치로 묶고 패딩합니다.
batch = data_collator(inputs)

# 결과를 출력합니다.
print(batch['input_ids'][15])


[210, 256, 256, 256, 210, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 110]
tensor([  101,  1037,  2047,  2259,  2103,  2522,  6491, 18903, 10727,  1010,
        20706,  2245,  2000,  2022,  1037,  2671,  3836,  1010,  2003,  3253,
         1037,  3105,  2000,  6570,  1996,  2336,  1997,  2019,  2789,  2647,
        21237,  1012,  1037, 17935,  4588,  2937,  1999,  2637,  2003, 20706,
         2245,  2000,  2022,  2019,  3834,  3836,  2011,  1037,  4387,  1997,
         2019,  2789,  2647, 21237,  1012,  2016,  2003,  4778,  2000,  2037,
         2406,  2006,  2008, 13534,  6772,  1998,  2003,  2356,  2000,  2022,
         1996, 14924,  1997,  1996, 21237,  1005,  1055,  2336,  1012,  2096,
         2045,  1010,  2016,  5363,  2000,  2530,  4697,  1996,  2878,  2406,
         1012,  1024,  1024,  4116,  2022,  8024,  7245,  1026, 14863,  8024,
         7245,  1030, 16260,  1012,  6187,  6895,  1012,  4012,  1028,   102,
            0,     0,     0,     0,     0,     0,     0,     

# 모델 정의

모델 설정과 모델 클래스 생성
- DistilBertForRegressionConfig
- DistilBertForRegression

In [27]:
from transformers import DistilBertConfig
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.distilbert.modeling_distilbert import DistilBertModel
from transformers.models.distilbert.modeling_distilbert import DistilBertPreTrainedModel

# DistilBertConfig를 상속받아 새로운 설정 클래스 정의
class DistilBertForRegressionConfig(DistilBertConfig):
    def __init__(self, user_num, factor_num, regressor_dropout, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.user_num = user_num
        self.factor_num = factor_num
        self.regressor_dropout = regressor_dropout

# DistilBertModel 바디에 Regression 헤드를 추가한 클래스 정의
class DistilBertForRegression(DistilBertPreTrainedModel):
    config_class = DistilBertForRegressionConfig

    def __init__(self, config):
        super().__init__(config)

        # 유저 임베딩 레이어
        self.embed_user = nn.Embedding(config.user_num, config.factor_num)
        # 모델 바디
        self.distilbert = DistilBertModel(config)
        # 예측 헤드
        self.regressor_dropout = nn.Dropout(config.regressor_dropout)
        self.regressor = nn.Linear(config.hidden_size + config.factor_num, config.num_labels)

        # 가중치를 로드하고 초기화
        self.init_weights()

    def forward(self, user_id=None, input_ids=None, attention_mask=None,
                labels=None, **kwargs):
        # 유저 임베딩 생성
        embed_user_output = self.embed_user(user_id)

        # distilbert 출력 생성하고, 맨 앞 토큰의 히든 임베딩을 조회
        distilbert_output = self.distilbert(input_ids=input_ids, attention_mask=attention_mask,
                               **kwargs)
        pooled_output = distilbert_output.last_hidden_state[:, 0]

        # 유저 임베딩과 히든 임베딩을 붙임
        concat = torch.cat((embed_user_output, pooled_output), -1) # (B, hidden_size+factor_num)

        # dropout 통과 및 회귀 결과 생성
        sequence_output = self.regressor_dropout(concat)
        logits = self.regressor(sequence_output).view(-1) # (B, 1) -> (B, )

        # 손실계산
        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)

## 설정 초기화

In [28]:
from transformers import AutoConfig

distil_config = AutoConfig.from_pretrained(pretrained_model_tag,
                                           num_labels=1
                                          )
distil_config.user_num=len(user_map)
distil_config.factor_num=32
distil_config.regressor_dropout=0.2

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## 모델 로딩

In [29]:
distil_model = DistilBertForRegression.from_pretrained(pretrained_model_tag, config=distil_config)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForRegression were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['embed_user.weight', 'regressor.bias', 'regressor.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 모델 아키텍쳐 관찰

In [30]:
distil_model

DistilBertForRegression(
  (embed_user): Embedding(943, 32)
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(

In [ ]:
# distil_model(**batch)

## 훈련 변수 설정

In [31]:
from transformers import TrainingArguments

num_epochs = 3
# 코랩에서 GPU 메모리 부족 에러가 나는 경우 batch_size를 16,32 등으로 줄여보자
batch_size = 64
logging_steps = len(train_ds) // batch_size
model_name = f"distilbert-fine-tuned"
training_args = TrainingArguments(
    output_dir=model_name, log_level="error", num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size, evaluation_strategy="epoch",
    save_steps=1e6, weight_decay=0.01, disable_tqdm=False,
    logging_steps=logging_steps, push_to_hub=False)

## 메트릭 정의 (MAE)

In [32]:
from sklearn.metrics import mean_absolute_error

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.squeeze()  # if necessary
    return {'mae': mean_absolute_error(labels, predictions)}

## 트레이너 생성

In [33]:
from transformers import Trainer

trainer = Trainer(model=distil_model, args=training_args,
                  data_collator=data_collator, compute_metrics=compute_metrics,
                  train_dataset=train_ds,
                  eval_dataset=validation_ds,
                  tokenizer=tokenizer)

In [34]:
distil_model.config.use_cache = False

# 훈련 실시

수행시간
- Colab T4: 1시간30분
- RTX 4090: 8분 30초
- Titan RTX: 27분 10초

In [35]:
train_result = trainer.train()

Epoch,Training Loss,Validation Loss,Mae
1,1.148600,1.086611,0.824717
2,1.042100,1.093218,0.823354
3,1.010400,1.051045,0.815507


## 훈련 결과 저장

In [36]:
trainer.save_model()

## 예측

In [37]:
predictions = trainer.predict(validation_ds)


In [38]:
sorted(zip(predictions.predictions, validation_ds['label'].numpy()), key=lambda x: x[0])[:100]

[(2.015118, 1.0),
 (2.0283957, 1.0),
 (2.0421073, 1.0),
 (2.0555747, 1.0),
 (2.072862, 1.0),
 (2.0764577, 2.0),
 (2.083135, 1.0),
 (2.096379, 1.0),
 (2.0964124, 1.0),
 (2.1029332, 1.0),
 (2.1062567, 1.0),
 (2.1074693, 1.0),
 (2.1076753, 2.0),
 (2.1086903, 1.0),
 (2.1101246, 1.0),
 (2.1105428, 3.0),
 (2.11114, 3.0),
 (2.1126335, 1.0),
 (2.1133187, 1.0),
 (2.1136005, 1.0),
 (2.1138985, 3.0),
 (2.115896, 2.0),
 (2.1173754, 3.0),
 (2.1182785, 1.0),
 (2.1198583, 3.0),
 (2.1213872, 3.0),
 (2.1227043, 1.0),
 (2.1235917, 1.0),
 (2.1236534, 4.0),
 (2.1240637, 3.0),
 (2.1246605, 1.0),
 (2.1251981, 1.0),
 (2.126465, 1.0),
 (2.1273055, 2.0),
 (2.129254, 1.0),
 (2.1319227, 4.0),
 (2.1344588, 2.0),
 (2.1365252, 1.0),
 (2.1387384, 1.0),
 (2.140148, 1.0),
 (2.1406133, 1.0),
 (2.141163, 2.0),
 (2.1429055, 1.0),
 (2.1440208, 2.0),
 (2.146645, 3.0),
 (2.150505, 4.0),
 (2.150654, 1.0),
 (2.1523228, 1.0),
 (2.1537778, 1.0),
 (2.1540647, 2.0),
 (2.1570039, 3.0),
 (2.1576006, 1.0),
 (2.1587727, 3.0),
 (2.161

In [39]:
sorted(zip(predictions.predictions, validation_ds['label'].numpy()), key=lambda x: x[0])[-100:]

[(4.5700493, 5.0),
 (4.571845, 5.0),
 (4.571994, 3.0),
 (4.572112, 4.0),
 (4.5721874, 5.0),
 (4.573191, 5.0),
 (4.573343, 2.0),
 (4.5737348, 5.0),
 (4.5738325, 5.0),
 (4.5738344, 5.0),
 (4.5767694, 4.0),
 (4.5786586, 4.0),
 (4.5794697, 1.0),
 (4.5795903, 4.0),
 (4.5802164, 5.0),
 (4.5803266, 4.0),
 (4.585622, 4.0),
 (4.5857916, 5.0),
 (4.5870004, 5.0),
 (4.588142, 4.0),
 (4.589179, 4.0),
 (4.590383, 2.0),
 (4.5951166, 4.0),
 (4.596179, 5.0),
 (4.5966406, 4.0),
 (4.598196, 4.0),
 (4.5988765, 5.0),
 (4.599804, 3.0),
 (4.600421, 5.0),
 (4.600767, 5.0),
 (4.6007824, 4.0),
 (4.600992, 5.0),
 (4.6019945, 5.0),
 (4.6036606, 4.0),
 (4.605335, 5.0),
 (4.605964, 3.0),
 (4.607424, 4.0),
 (4.6074276, 3.0),
 (4.607429, 4.0),
 (4.6076627, 5.0),
 (4.6093535, 3.0),
 (4.609736, 3.0),
 (4.610181, 4.0),
 (4.6103477, 3.0),
 (4.610391, 5.0),
 (4.611059, 4.0),
 (4.615364, 5.0),
 (4.6154904, 4.0),
 (4.6175175, 4.0),
 (4.6196184, 5.0),
 (4.6203985, 5.0),
 (4.6216116, 5.0),
 (4.6222067, 5.0),
 (4.6226206, 5.0)